In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.tokenize import word_tokenize

In [2]:
sys.path.append('../scripts')

In [3]:
df = pd.read_csv('../Scripts/telegram_data.csv')

In [4]:
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1378,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:43+00:00,photos\@kuruwear_1378.jpg
1,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1377,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:35+00:00,photos\@kuruwear_1377.jpg
2,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1376,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:22+00:00,photos\@kuruwear_1376.jpg
3,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1375,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:04+00:00,photos\@kuruwear_1375.jpg
4,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1374,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-14 07:11:18+00:00,photos\@kuruwear_1374.jpg
...,...,...,...,...,...,...
2379,Et Online Shopping,@Etonline_shopping,15996,Lmn🪝Emporio Armani 🪝\n🪝 Noble Atmosphere \n🪝 L...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15996.jpg
2380,Et Online Shopping,@Etonline_shopping,15995,👑ROLEX 🪄\n🦚 Noble Atmosphere \n🦚 Luxury Made \...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15995.jpg
2381,Et Online Shopping,@Etonline_shopping,15994,NaN,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15994.jpg
2382,Et Online Shopping,@Etonline_shopping,15993,NaN,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15993.jpg


In [4]:
print(df.describe())

                 ID
count   2384.000000
mean   12549.365352
std     7175.656707
min        1.000000
25%     2260.000000
50%    16642.500000
75%    17493.250000
max    18750.000000


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384 entries, 0 to 2383
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Channel Title     2384 non-null   object
 1   Channel Username  2384 non-null   object
 2   ID                2384 non-null   int64 
 3   Message           994 non-null    object
 4   Date              2384 non-null   object
 5   Media Path        2245 non-null   object
dtypes: int64(1), object(5)
memory usage: 111.9+ KB
None


In [5]:
print("Checking for NaN values in the 'Message' column:")
nan_count = df['Message'].isnull().sum()
print(f"Number of NaN values in 'Message' column: {nan_count}")

Checking for NaN values in the 'Message' column:
Number of NaN values in 'Message' column: 1390


In [6]:
df = df.dropna(subset=['Message'])

# Print the shape of the dataset after dropping NaN values in the "Message" column
print(f"Dataset shape after dropping NaN values in 'Message' column: {df.shape}")

Dataset shape after dropping NaN values in 'Message' column: (994, 6)


In [7]:
df.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1378,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:43+00:00,photos\@kuruwear_1378.jpg
1,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1377,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:35+00:00,photos\@kuruwear_1377.jpg
2,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1376,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:22+00:00,photos\@kuruwear_1376.jpg
3,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1375,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-21 06:16:04+00:00,photos\@kuruwear_1375.jpg
4,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1374,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n 🔥...,2024-09-14 07:11:18+00:00,photos\@kuruwear_1374.jpg


In [8]:
message_df=df['Message']
message_df

0       100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n     🔥...
1       100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n     🔥...
2       100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n     🔥...
3       100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n     🔥...
4       100% ንፁ ቆዳ የተሰሩ ጫማዎች\n🇪🇹 የሀገራችን ምርት 🇪🇹\n     🔥...
                              ...                        
2375    🌷DIOR 🌷\n🌷JOY 🌷\n🌷 100Ml \n🌷 PERFUME (ሽቶ)\n🌷 F...
2376    👨‍🦱 ሽበት በአጭር ጊዜ ማጥፋት እንደሚችሉ ያቃሉ\n🧑‍🦱 እንግዱያው Pa...
2377    🪄T55 PRO MAX🪄\n💭 Scroll Working \n💭 1.75inch D...
2379    Lmn🪝Emporio Armani 🪝\n🪝 Noble Atmosphere \n🪝 L...
2380    👑ROLEX 🪄\n🦚 Noble Atmosphere \n🦚 Luxury Made \...
Name: Message, Length: 994, dtype: object

In [9]:
import re

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["                           # Start character class
        "\U0001F600-\U0001F64F"       # Emoticons
        "\U0001F300-\U0001F5FF"       # Miscellaneous Symbols and Pictographs
        "\U0001F680-\U0001F6FF"       # Transport and Map Symbols
        "\U0001F1E0-\U0001F1FF"       # Flags (iOS)
        "\U00002702-\U000027B0"       # Dingbats
        "\U000024C2-\U0001F251"       # Enclosed characters
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Apply the function to the 'Message' column
df['Message'] = df['Message'].apply(remove_emojis)

# Display the updated DataFrame
print(df.head())


# # Example usage:
# text = "Here is some text with an emoji 😊 and another one 🎉."
# print(remove_emojis(text))  # Output: Here is some text with an emoji  and another one .


         Channel Title Channel Username    ID  \
0  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1378   
1  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1377   
2  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1376   
3  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1375   
4  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1374   

                                             Message  \
0  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
1  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
2  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
3  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2800 ...   
4  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   

                        Date                 Media Path  
0  2024-09-21 06:16:43+00:00  photos\@kuruwear_1378.jpg  
1  2024-09-21 06:16:35+00:00  photos\@kuruwear_1377.jpg  
2  2024-09-21 06:16:22+00:00  photos\@kuruwear_1376.jpg  
3  2024-09-21 06:16:04+00:00  photos\@kuruwear_1375.jpg  
4  2024-09-14 07:11:18+00:00  photos\@kuruwear_1374.jpg  


C:\Users\amanu\AppData\Local\Temp\ipykernel_5604\206848078.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Message'] = df['Message'].apply(remove_emojis)


In [10]:
df.to_csv('../data/cleaned_data.csv')

In [11]:
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1378,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-21 06:16:43+00:00,photos\@kuruwear_1378.jpg
1,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1377,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-21 06:16:35+00:00,photos\@kuruwear_1377.jpg
2,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1376,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-21 06:16:22+00:00,photos\@kuruwear_1376.jpg
3,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1375,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2800 ...,2024-09-21 06:16:04+00:00,photos\@kuruwear_1375.jpg
4,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1374,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-14 07:11:18+00:00,photos\@kuruwear_1374.jpg
...,...,...,...,...,...,...
2375,Et Online Shopping,@Etonline_shopping,16000,DIOR \nJOY \n 100Ml \n PERFUME (ሽቶ)\n For Her/...,2022-12-07 10:05:40+00:00,photos\@Etonline_shopping_16000.jpg
2376,Et Online Shopping,@Etonline_shopping,15999,‍🦱 ሽበት በአጭር ጊዜ ማጥፋት እንደሚችሉ ያቃሉ\n🧑‍🦱 እንግዱያው Pan...,2022-12-07 09:52:57+00:00,photos\@Etonline_shopping_15999.jpg
2377,Et Online Shopping,@Etonline_shopping,15998,🪄T55 PRO MAX🪄\n Scroll Working \n 1.75inch Dis...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15998.jpg
2379,Et Online Shopping,@Etonline_shopping,15996,Lmn🪝Emporio Armani 🪝\n🪝 Noble Atmosphere \n🪝 L...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15996.jpg


In [12]:
import re

# Comprehensive function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # Chinese characters
        "\U00002702-\U000027B0"  # various symbols
        "\U00002702-\U000027B0"  # dingbats
        "\U000024C2-\U0001F251"  # enclosed characters
        "\U0001F926-\U0001FA9F"  # additional emoticons
        "\U0001F1F2-\U0001F1F4"  # additional flags
        "\U0001F918-\U0001F9C0"  # more symbols
        "\U0000200D"             # zero-width joiner
        "\U000023CF-\U0001F3FB"  # additional pictographs
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)
# Apply the function to the 'Message' column
df['Message'] = df['Message'].apply(remove_emojis)

# Display the updated DataFrame
print(df.head())

# # Test the updated function on a string containing emojis
# text_with_emojis = "This is a test 😊 with some 🎉 emojis and symbols 📱!"
# print(remove_emojis(text_with_emojis))


         Channel Title Channel Username    ID  \
0  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1378   
1  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1377   
2  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1376   
3  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1375   
4  ኩሩ/kuru/ 🇪🇹 wear ®️        @kuruwear  1374   

                                             Message  \
0  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
1  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
2  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
3  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2800 ...   
4  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   

                        Date                 Media Path  
0  2024-09-21 06:16:43+00:00  photos\@kuruwear_1378.jpg  
1  2024-09-21 06:16:35+00:00  photos\@kuruwear_1377.jpg  
2  2024-09-21 06:16:22+00:00  photos\@kuruwear_1376.jpg  
3  2024-09-21 06:16:04+00:00  photos\@kuruwear_1375.jpg  
4  2024-09-14 07:11:18+00:00  photos\@kuruwear_1374.jpg  


C:\Users\amanu\AppData\Local\Temp\ipykernel_5604\2754364963.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Message'] = df['Message'].apply(remove_emojis)


In [13]:
df.to_csv('../data/clean_data.csv')

In [14]:
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1378,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-21 06:16:43+00:00,photos\@kuruwear_1378.jpg
1,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1377,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-21 06:16:35+00:00,photos\@kuruwear_1377.jpg
2,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1376,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-21 06:16:22+00:00,photos\@kuruwear_1376.jpg
3,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1375,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2800 ...,2024-09-21 06:16:04+00:00,photos\@kuruwear_1375.jpg
4,ኩሩ/kuru/ 🇪🇹 wear ®️,@kuruwear,1374,100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n 2400 ...,2024-09-14 07:11:18+00:00,photos\@kuruwear_1374.jpg
...,...,...,...,...,...,...
2375,Et Online Shopping,@Etonline_shopping,16000,DIOR \nJOY \n 100Ml \n PERFUME (ሽቶ)\n For Her/...,2022-12-07 10:05:40+00:00,photos\@Etonline_shopping_16000.jpg
2376,Et Online Shopping,@Etonline_shopping,15999,ሽበት በአጭር ጊዜ ማጥፋት እንደሚችሉ ያቃሉ\n እንግዱያው Pansly n...,2022-12-07 09:52:57+00:00,photos\@Etonline_shopping_15999.jpg
2377,Et Online Shopping,@Etonline_shopping,15998,T55 PRO MAX\n Scroll Working \n 1.75inch Displ...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15998.jpg
2379,Et Online Shopping,@Etonline_shopping,15996,LmnEmporio Armani \n Noble Atmosphere \n Luxur...,2022-12-07 09:52:56+00:00,photos\@Etonline_shopping_15996.jpg


In [18]:
import pandas as pd
import re

# Load the dataset
df = pd.read_csv('../data/clean_data.csv')

# Function to remove English words from a message
def remove_english_words(message):
    # Check if the message is a string
    if isinstance(message, str):
        # Remove any words containing English letters (A-Z, a-z)
        cleaned_message = re.sub(r'\b[a-zA-Z]+\b', '', message)
        # Remove any extra whitespace that might result from the removal
        cleaned_message = re.sub(r'\s+', ' ', cleaned_message).strip()
        return cleaned_message
    return message  # or return '' if you want to replace non-string values with an empty string

# Apply the function to the 'Message' column
df['Cleaned_Message'] = df['Message'].apply(remove_english_words)

In [26]:
# Display the cleaned DataFrame
print(df[['Message', 'Cleaned_Message']].head())

# Save the cleaned DataFrame if needed
df.to_csv('../data/cleaned_data.csv', index=False)

                                             Message  \
0  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
1  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
2  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   
3  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2800 ...   
4  100% ንፁ ቆዳ የተሰሩ ጫማዎች\n የሀገራችን ምርት \n     2400 ...   

                                     Cleaned_Message  
0  100% ንፁ ቆዳ የተሰሩ ጫማዎች የሀገራችን ምርት 2400 ብር ብቻ 091...  
1  100% ንፁ ቆዳ የተሰሩ ጫማዎች የሀገራችን ምርት 2400 ብር ብቻ 091...  
2  100% ንፁ ቆዳ የተሰሩ ጫማዎች የሀገራችን ምርት 2400 ብር ብቻ 091...  
3  100% ንፁ ቆዳ የተሰሩ ጫማዎች የሀገራችን ምርት 2800 ብር ብቻ 091...  
4  100% ንፁ ቆዳ የተሰሩ ጫማዎች የሀገራችን ምርት 2400 ብር ብቻ 091...  


In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv('../data/cleaned_data.csv')


In [3]:
from amseg.amharicSegmenter import AmharicSegmenter

In [4]:
import pandas as pd

# Load your dataset
df = pd.read_csv('../data/cleaned_data.csv')

In [ ]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [ ]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [ ]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [ ]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [ ]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")


In [ ]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('../data/final_tokens_labels.csv', index=False)

In [9]:
import pandas as pd
from amseg.amharicSegmenter import AmharicSegmenter

# Load your dataset
df = pd.read_csv('../data/cleaned_data.csv')

# Specify the column containing Amharic text (replace 'text_column' with the actual column name)
amharic_text_column = 'Message'

# Initialize Amharic segmenter
sent_punct = []  # Define your sentence punctuation rules if needed
word_punct = []  # Define word punctuation rules if needed
segmenter = AmharicSegmenter(sent_punct, word_punct)

# Function to tokenize Amharic text
def tokenize_amharic_text(text):
    # Check if text is not NaN or float
    if pd.isna(text) or not isinstance(text, str):
        return ''
    return ' '.join(segmenter.amharic_tokenizer(text))  # Tokenize and join tokens with space

# Apply tokenization to the dataset (convert to string and handle NaN)
df[amharic_text_column] = df[amharic_text_column].astype(str)  # Convert column to string
df['tokenized_text'] = df[amharic_text_column].apply(tokenize_amharic_text)



In [10]:
# Save the tokenized data to a new CSV file
df.to_csv('../data/tokenized_cleaned_data.csv', index=False)

print("Tokenization complete. The tokenized dataset has been saved to '../data/tokenized_cleaned_data.csv'.")

Tokenization complete. The tokenized dataset has been saved to '../data/tokenized_cleaned_data.csv'.
